In [57]:
import pandas as pd
import numpy as np
from itertools import chain
from numpy import load, save
import h5py
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report, confusion_matrix
from gensim.models import FastText
from gensim.utils import tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from nltk import PorterStemmer, word_tokenize

In [2]:
project = pd.read_csv ('C:/Users/yh448/OneDrive - University of Cambridge/Name_text/paper/full_short_.csv')
project = project.rename(columns={"Logic_y": "Succ_logic", "Lag_y": "Succ_lag", "Logic_x": "Pred_logic", "Lag_x": "Pred_lag"})
p_id = project.drop_duplicates (subset = 'Project_ID')
p_id['Project_ID'].astype('int32')
id_list = p_id['Project_ID'].tolist()
print (len(id_list), type(id_list[1]))

32 <class 'int'>


In [22]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(', ')))

def full_graph (pred, logic, lag, df):
    df[pred] = df[pred].astype(str)
    df[logic] = df[logic].astype(str)
    df[lag] = df[lag].astype(str)
    a = chainer(df[pred])
    b = chainer(df[logic])
    c = chainer(df[lag])
    lens = df[pred].str.split(', ').map(len)
    graph = pd.DataFrame({pred: a, 'Clean_task_code': np.repeat(df['Clean_task_code'], lens), 
                      logic: b, lag: c,})
    return graph

def weight (logic, df, name):
    ls_l = df[logic].tolist()
    ls = []
    mid = df.groupby(logic).count()[['Clean_task_code']]
    for i in ls_l:
        if i == 'PR_FF':
            ls.append ((mid.loc['PR_FF', 'Clean_task_code'])/len(full_))
        if i == 'PR_SS':
            ls.append ((mid.loc['PR_SS', 'Clean_task_code'])/len(full_))
        if i == 'PR_FS':
            ls.append ((mid.loc['PR_FS', 'Clean_task_code'])/len(full_))
        if i == 'PR_SF':
            ls.append ((mid.loc['PR_SF', 'Clean_task_code'])/len(full_))
        if i == 'None':
            ls.append ((mid.loc['None', 'Clean_task_code'])/len(full_))
        if i == 'nan':
            ls.append ((mid.loc['nan', 'Clean_task_code'])/len(full_))
    df[name] = ls
    return df
def pend_node(name1, df, name2):
    name_ls = df[name1].tolist()
    ls1 = []
    for i in name_ls:
        if i != 'nan':
            num = df.loc[df['Clean_task_code']==i, ['Graph_x']].median()['Graph_x']
        else: num = int(10000)
        ls1.append (num)
    df[name2] = ls1
    df[name2] = df[name2].fillna(10000)
    return df
def group_label(name1, df, name2):
    name_ls = df[name1].tolist()
    ls1 = []
    for i in name_ls:
        g = df.loc[df['Clean_task_code']==i,]
        if g.shape[0] != 0:
            num = g.Foundation_detail.any()
        else: num = 'None'
        ls1.append (num)
    df[name2] = ls1
    return df
#pred is row, succ is column, loc = row*dim+column
def adjacency_matrix (namep, names, graph_1, weight_1, weight_2, df):
    name1_ls = df[namep].tolist()
    graph1 = df[graph_1].tolist()
    weight1 = df[weight_1].tolist()
    name2_ls = df[names].tolist()
    weight2 = df[weight_2].tolist()
    x = df.drop_duplicates(subset = 'Clean_task_code')
    length = x.shape[0]
    
    ls1,ls2,ls3,ls4 = [],[],[],[]
    for i, j, m in zip(name1_ls, graph1, weight1):
        if i == 10000:
            ls1.append (0)
            ls2.append (0)
        else: 
            ls1.append(i*length+j)
            ls2.append(m)
    for i, j, k in zip(name2_ls, graph1, weight2):
        if i == 10000:
            ls3.append (0)
            ls4.append (0)
        else: 
            ls3.append(i+j*length)
            ls4.append(k)         
            
    x = np.zeros((length, length))
    np.put(x, ls1, ls2)
    np.put(x, ls3, ls4)
    np.fill_diagonal(x, 1)
    return x
def degree_matrix(matrix):
    ls1 = []
    length = len(matrix)
    x = np.zeros((length, length))
    for i in range (length):
        ls1.append (np.sum(matrix[i]))
    np.fill_diagonal(x, ls1)
    return x
def normal_matrix (degree_m, ad_m):
    x = np.power(degree_m.diagonal(), -0.5)
    y = np.zeros((len(degree_m), len(degree_m)))
    np.fill_diagonal(y, x)
    mid = np.dot(y, ad_m)
    a_delta = np.dot (mid, y)
    return a_delta 

lemmatizer = WordNetLemmatizer()
def prep (ls):
    test_token, test_lemma, test_return = [], [], []
    for i in range (len (ls)):
        token = tokenize(str(ls[i]))
        test_token.append (list(token))  

    for i in range (len (test_token)):
        x = lemmatizer.lemmatize(str(test_token[i]))
        x1 = eval('' + x + '')
        test_lemma.append (x1)
    
    for i in range (len (test_lemma)):
        test_return.append (' '.join(test_lemma[i]))
        
    return test_return
def token (ls):
    ps = PorterStemmer()
    token_word = []
    for i in range (len(ls)):
        ls1 = word_tokenize(str(ls[i]))
        ls2 = []
        for word in ls1:
            word = word.lower()
            word = ps.stem(word)   
            ls2.append (word)
        token_word.append (ls2)
    return token_word 

In [105]:
test_ = pd.read_csv ('C:/Users/yh448/OneDrive - University of Cambridge/Name_text/paper/test_head.csv')
init = np.zeros((1,1))
for i in id_list_:
    x2 = project.loc[project['Project_ID']== i,]
    empty_pred = full_graph('Predecessor', 'Pred_logic', 'Pred_lag', x2)
    empty_succ = full_graph('Successor', 'Succ_logic', 'Succ_lag',x2)
    empty_succ = empty_succ.mask(empty_succ.eq('None')).dropna()
    empty_pred = empty_pred.mask(empty_pred.eq('None')).dropna()
    x2_others = x2.loc[:,['Clean_task_code','Duration', 'Project_ID','Task_name', 'Clean_task_name','WBS', 'Graph_x','r_length', 'r_pos', 'Foundation_detail']]
    full = pd.merge (empty_pred, empty_succ, on = ('Clean_task_code'))
    full_ = pd.merge (full, x2_others, on = 'Clean_task_code')
    full1 = weight ('Pred_logic', full_, 'Weight_pred')
    full2 = weight ('Succ_logic', full1, 'Weight_succ')
    graph1 = pend_node('Predecessor', full2, 'Pred_graph')
    graph2 = pend_node('Successor', graph1, 'Succ_graph')
    label1 = group_label('Predecessor', graph2, 'Pred_found')
    label2 = pend_node('Successor', label1, 'Succ_found')
    label2['Pred_graph'] = np.where(label2.Pred_found.any() == label2.Foundation_detail.any() != 'None' , label2['Graph_x'], label2['Pred_graph'])
    label2['Succ_graph'] = np.where(label2.Succ_found.any() == label2.Foundation_detail.any() != 'None' , label2['Graph_x'], label2['Succ_graph'])
    test_ = test_.append (label2)
    #amatrix = adjacency_matrix ('Pred_graph', 'Succ_graph', 'Graph_x', 'Weight_pred', 'Weight_succ', label2)
    #print (amatrix.shape)

C:\Users\yh448\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\yh448\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yh448\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [23]:
#train_name = test_['Clean_task_name'].tolist()
#train_name_ = token(prep(train_name))
#trained_model = FastText(train_name_, size = 60, min_count = 1, negative = 7, workers = 5)
x = test_.drop_duplicates(subset = 'Clean_task_code')
tokens = x['Clean_task_name'].tolist()
token_ = token(prep(tokens))

In [26]:
def feature_matrix (df):
    x = df.drop_duplicates(subset = 'Clean_task_code')
    empty = np.zeros(60).reshape(1,60)
    ls = x['Clean_task_name'].tolist()
    ls_token = token(prep(ls))
    length_curr = x['r_length'].replace('None',0).tolist()
    pos_curr  = x['r_pos'].replace('None',0).tolist()
    x['Duration'] = x['Duration'].astype(int)
    a = x['Duration'].to_numpy()
    dur_curr = (np.log(a+1)).reshape(len(ls),1)
    for i in range (len(ls)):
        n = ' '.join(ls_token[i])
        m = trained_model.wv[n].reshape(1,60)
        empty = np.append (empty, m, axis = 0)
    empty_ = np.delete(empty, 0, axis = 0)
    len_curr = np.asarray(length_curr).reshape(len(ls),1)
    po_curr = np.asarray(pos_curr).reshape(len(ls),1)
    empty_ = np.append (empty_, len_curr, axis = 1)
    empty_ = np.append (empty_, po_curr, axis = 1)
    empty_ = np.append (empty_, dur_curr, axis = 1)
    empty_ = empty_.astype(float)
    return empty_ 

In [103]:
id_list_ = [3148, 4019, 4459, 5046, 6621, 34143, 41214, 75020, 93867, 93917, 96831, 99673, 100785, 100835, 101597, 118744, 118944, 118994, 119044, 119094, 119299, 121395, 131325, 140297, 142884]

In [104]:
init = np.zeros((1,1))
f_init = np.zeros((1,63))
for i in id_list_:
    x2 = test_.loc[test_['Project_ID']==i,]
    amatrix = adjacency_matrix ('Pred_graph', 'Succ_graph', 'Graph_x', 'Weight_pred', 'Weight_succ', x2)
    A = init.shape[0]
    B = amatrix.shape[0]
    init = np.block([[init,np.zeros((A,B))],[np.zeros((B,A)), amatrix]])
    feature = feature_matrix(x2)
    f_init = np.append (f_init, feature, axis=0)
    print(x2.shape, amatrix.shape, init.shape, f_init.shape, feature.dtype, i)
init_ = np.delete(init, 0, 0)
init_f = np.delete(init_, 0, 1)
degree_m = degree_matrix (init_f)
normal_m = normal_matrix (degree_m, init_f)
feature_m = np.delete(f_init, 0, 0)
train_in = np.dot (normal_m, feature_m)

C:\Users\yh448\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(664, 23) (255, 255) (256, 256) (256, 63) float64 3148
(56, 23) (24, 24) (280, 280) (280, 63) float64 4019
(261, 23) (81, 81) (361, 361) (361, 63) float64 4459
(538, 23) (136, 136) (497, 497) (497, 63) float64 5046
(506, 23) (198, 198) (695, 695) (695, 63) float64 6621
(3891, 23) (1463, 1463) (2158, 2158) (2158, 63) float64 34143
(380, 23) (140, 140) (2298, 2298) (2298, 63) float64 41214
(297, 23) (108, 108) (2406, 2406) (2406, 63) float64 75020
(971, 23) (300, 300) (2706, 2706) (2706, 63) float64 93867
(881, 23) (281, 281) (2987, 2987) (2987, 63) float64 93917
(344, 23) (171, 171) (3158, 3158) (3158, 63) float64 96831
(1101, 23) (457, 457) (3615, 3615) (3615, 63) float64 99673
(1000, 23) (311, 311) (3926, 3926) (3926, 63) float64 100785
(1051, 23) (326, 326) (4252, 4252) (4252, 63) float64 100835
(843, 23) (291, 291) (4543, 4543) (4543, 63) float64 101597
(1710, 23) (484, 484) (5027, 5027) (5027, 63) float64 118744
(1901, 23) (512, 512) (5539, 5539) (5539, 63) float64 118944
(1181, 23

In [106]:
test_nodup = test_.drop_duplicates(subset = ('Clean_task_code', 'Project_ID'))
test_nodup['Foundation_detail'] = test_nodup['Foundation_detail'].astype('category')
test_nodup["Ont_label"] = test_nodup['Foundation_detail'].cat.codes
print (test_.shape, test_nodup.shape)

(30157, 23) (9016, 24)


C:\Users\yh448\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yh448\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
batch_size = [20, 40, 60, 70, 80]
epochs = [60, 70, 80, 90, 100]
optimizer = ['SGD']
neurons = [50,70,100,120,150]

def create_model(optimizer='sgd', neurons = 20): 
    gcn_model = models.Sequential()
    gcn_model.add(layers.Dense(neurons, activation='relu', input_dim = 63))
    gcn_model.add(layers.Dense(neurons, activation='softmax'))
    gcn_model.add(layers.Dense(neurons, activation='relu'))
    gcn_model.add(layers.Dense(neurons, activation='softmax'))
    gcn_model.add(layers.Dense(neurons, activation='relu'))
    gcn_model.add(layers.Dense(neurons, activation='softmax'))
    gcn_model.add(layers.Dense(neurons, activation='relu'))
    gcn_model.add(layers.Dense(neurons, activation='softmax'))
    gcn_model.add(layers.Dense(neurons, activation='relu'))
    gcn_model.add(layers.Dense(neurons, activation='softmax'))
    gcn_model.add(layers.Dense(1))
    #optimizer = tf.keras.optimizers.SGD(lr=learn_rate, momentum=momentum)
    gcn_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return gcn_model

gcn_model = KerasClassifier(build_fn = create_model, verbose = 0)
param_grid_1 = dict(batch_size = batch_size, epochs = epochs, optimizer=optimizer, 
                    neurons = neurons)#
grid_1 = GridSearchCV(estimator = gcn_model, param_grid=param_grid_1, 
                      n_jobs=-1, cv=5, scoring='f1_macro')
label_x = test_nodup['Ont_label'].to_numpy()
grid_result = grid_1.fit(train_in, label_x)
best = grid_result.best_estimator_
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print (classification_report(label_x, best.predict(train_in)))
print (confusion_matrix(label_x, best.predict(train_in)))

C:\Users\yh448\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [72]:
print (classification_report(label_x, best.predict(train_in)))
print (confusion_matrix(label_x, best.predict(train_in)))

              precision    recall  f1-score   support

           0       0.00      1.00      0.01         9
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00      2238
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00         5

    accuracy                           0.00      2297
   macro avg       0.00      0.20      0.00      2297
weighted avg       0.00      0.00      0.00      2297

[[   9    0    0    0    0]
 [  35    0    0    0    0]
 [2238    0    0    0    0]
 [  10    0    0    0    0]
 [   5    0    0    0    0]]


In [109]:
test_nodup.groupby('Foundation_detail').count()[['Ont_label']]

,Ont_label
Foundation_detail,
Base slab,9
Floor slab,6
Foundation,11
General,2
Lift pit,37
Mat,28
None,8869
Pad,2
Parapet Wall,8
